# MP2- Traffic Sign Classifier
Before you start, please read through this tutorial of Pytorch: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py.
The basic structure of this MP will be very similar to the example code in the tutorial. But your neural network model need to be more complicated. 

If any packege is not installed, you shall be able to install them through pip3 install. 

In [1]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage import io, transform
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, sampler
import torchvision
from torchvision import transforms, utils
import torchvision.utils as vutils
import torchvision.datasets as dset
from torch.autograd import Variable

Set up the parameters. If you have GPU, change the num_gpu. 

In [2]:
# Number of workers for dataloader
workers = 4

# Batch size during training
batch_size = 128

# Spatial size of training images. The image size of original images vary from 15 to 250. 
#But to train our CNN we must have the fixed size for the inputs.
#All images will be resized to this size using a transformer.
image_size = 32

# Number of training epochs
num_epochs = 15

#number of training classes
num_classes = 43

# Learning rate for optimizers
learning_rate = 0.002

# Number of GPUs available. Use 0 for CPU mode.
num_gpu = 1

# Set random seed for reproducibility
manualSeed = 999
torch.manual_seed(manualSeed)

### Data Preprocessing 
To improve the performance of our CNN, we need to use some data augmentation techniques. We can change the orientation, location, scale, saturation or brightness of the original image to modify the original images or create new training images. 

The easiest way to go is openCV. If you are having problem with ppm format, you can convert the images to other image format (PNG, JPG...) using pillow packege: https://pillow.readthedocs.io/en/latest/handbook/tutorial.html

You are reqiured to test at least 3 techniques and compare the effectiveness(the accuracy improvement). For each technique, you need to generate new images for at least one class of images. For instance, if you originally have 300 training images in class 01, then you need to generate 300 new images and save them into the same folder as old images. Note that it may be helpful to name the new images properly, so if you find your technique is not helpful, you can remove them easier. <br>
You can skip this part and try to have a working neural network first, then come back here to finish it. 

In [138]:
def transform(img_dir, method):
    for subdir, dirs, imgs in os.walk(img_dir):
        for img_name in imgs:
            if img_name.endswith('.ppm'):
                img = cv2.imread(os.path.join(img_dir, img_name), 1)
                # scaling images
                if method == 'scale':
                    choices = [0, 1]
                    choice = random.choice(choices)
                    # enlarge or shrink the image by factor of 2
                    rows, cols, channel = img.shape
                    # print(rows, cols)
                    if choice == 0:
                        res = cv2.resize(img, (rows*2, cols*2), interpolation = cv2.INTER_CUBIC)
                    else:
                        res = cv2.resize(img, (int(rows/2), int(cols/2)), interpolation = cv2.INTER_AREA)

                elif method == 'rotate':
                    choices = [0, 1, 2]
                    choice = random.choice(choices)
                    rows, cols, channel = img.shape
                    if choice == 0:
                        M = cv2.getRotationMatrix2D((cols/2, rows/2), 90, 1)
                    elif choice == 1:
                        M = cv2.getRotationMatrix2D((cols/2, rows/2), 180, 1)
                    else:
                        M = cv2.getRotationMatrix2D((cols/2, rows/2), 270, 1)
                    res = cv2.warpAffine(img, M, (cols, rows))

                elif method == 'flip':
                    choices = [0, 1, 2]
                    choice = random.choice(choices)
                    if choice == 0:
                        res = cv2.flip(img, 0)
                    elif choice == 1:
                        res = cv2.flip(img, 1)
                    else:
                        res = cv2.flip(img, -1)
                name, extension = os.path.split(img_name)
                res_name = name + 'trans' + extension
                cv2.imwrite(os.path.join(img_dir, res_name), res)

scale_dir = 'GTSRB/00024_scale/'
rotate_dir = 'GTSRB/00024_rotate/'
flip_dir = 'GTSRB/00024_flip/'

#transform(scale_dir, 'scale')
#transform(rotate_dir, 'rotate')
#transform(flip_dir, 'flip')
print('...')

...


### Load the data
Load the dataset into PyTorch. Assign the path to the training and testing dataset you downloaded previously to the variables "trainingClassifierRoot" and "testClassifierRoot" below. In the beginning you can just use the original data downloaded from website. Later on you will need to process the image before load them.  

In [139]:
##TODO
trainingClassifierRoot = 'GTSRB/Final_Training/Images'
#trainingClassifierRoot = 'GTSRB/Final_Training_flip/Images'
#testClassifierRoot = '/home/peixin/mp2_data/Online-Test-sort/'
testClassifierRoot = 'GTSRB/Online-Test-Images-Sorted'
####
normImgTensor = transforms.Normalize(mean=np.zeros(3), std=np.ones(3))

# Create the dataset
trainClassifierDataset = dset.ImageFolder(root=trainingClassifierRoot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
							   normImgTensor
                           ]))

testClassifierDataset = dset.ImageFolder(root=testClassifierRoot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
							   normImgTensor
                           ]))

# Create the dataloader
trainClassifierLoader = torch.utils.data.DataLoader(trainClassifierDataset, batch_size=batch_size, shuffle=True, num_workers=workers)
testClassifierLoader = torch.utils.data.DataLoader(testClassifierDataset, batch_size=batch_size, shuffle=True, num_workers=workers)

# Decide which device(GPU or CPU) we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and num_gpu > 0) else "cpu")

print ('...')

...


### Design the model
In this part you will design the structure of CNN. The basic syntax is the same as the toy example in PyTorch tutorial. But you definitely need some more complex models to meet the reqiurements. You need to try at least 3 different structures and report their accuracy and running time (how long it would take to go through the test set). The best one is required to achieve 97% accuracy. Describe the CNN structures and compare the results in your report. Feel free to use any existing structure you find on Internet, but citation in your report is needed.<br>
Here is a NN structure that has 98% accuracy. You can start from there: https://chatbotslife.com/german-sign-classification-using-deep-learning-neural-networks-98-8-solution-d05656bf51ad<br>
If you really want to get higher accuracy, here are some state-of-the-art papers for your reference: 
<li>https://arxiv.org/abs/1511.02992</li>
<li>https://www.sciencedirect.com/science/article/pii/S0893608018300054?via%3Dihub</li>
<li>http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf</li>

In [140]:
class TrafficSignClassifier(nn.Module):
    def __init__(self, num_gpu):
        super(TrafficSignClassifier, self).__init__()
        self.ngpu = num_gpu
       
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, padding=(2,2))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, padding=(2,2))
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=(2,2))

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=(2,2))
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, padding=(2,2))
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5, padding=(2,2))

        self.pool = nn.MaxPool2d(2, 2)

        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(128)
        self.bn5 = nn.BatchNorm2d(256)
        self.bn6 = nn.BatchNorm2d(512)
        
        self.relu = nn.ReLU(inplace = True)
        self.linear = nn.Linear(4*4*64, num_classes)
        ####
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x  = self.pool(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64 * 4 * 4)
        x = self.linear(x) 
        return x
	
class TrafficSignClassifier1(nn.Module):
	def __init__(self, num_gpu):
		super(TrafficSignClassifier1, self).__init__()
		self.ngpu = num_gpu
		
		self.conv0 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=1)
		
		self.conv1a = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding = (1, 1))
		self.conv1b = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
		
		self.conv2a = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding = (1, 1))
		self.conv2b = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding = (1, 1))
		
		self.conv3a = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding = (1, 1))
		self.conv3b = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding = (1, 1))
		
		self.pool = nn.MaxPool2d(3, 3)
		
		self.dropout = nn.Dropout2d(p = 0.2)
		
		self.linear1 = nn.Linear(32*10*10, 128)
		self.linear2 = nn.Linear(64*3*3, 128)
		self.linear3 = nn.Linear(128, num_classes)
		
	def forward(self, x):
		x = self.conv0(x)
		x = self.conv1a(x)
		x = self.conv1b(x)
		x = self.pool(x)
		x1 = self.dropout(x)

		x = self.conv2a(x1)
		x = self.conv2b(x)
		x = self.pool(x)
		x2 = self.dropout(x)
	
		x = self.conv3a(x2)
		x = self.conv3b(x)
		x = self.pool(x)
		x3 = self.dropout(x)

		x1 = x1.view(-1, 32*10*10)
		x1 = self.linear1(x1)
		x2 = x2.view(-1, 64*3*3)
		x2 = self.linear2(x2)

		x3 = x3.view(-1, 128*1*1)
		x3 = x1 + x2 + x3
		x3 = self.linear3(x3)
		
		return x3

class TrafficSignClassifier2(nn.Module):
	def __init__(self, num_gpu):
		super(TrafficSignClassifier2, self).__init__()
		self.ngpu = num_gpu
		
		self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=1)
		
		# self.conv2 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
		self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding = (1, 1))
		
		self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding = (2, 2))
		
		self.pool4 = nn.MaxPool2d(3, 3)
		
		self.linear1 = nn.Linear(32*(32*32*3 + 10*10), 128)
		self.linear2 = nn.Linear(32*32*32, num_classes)
		self.linear4 = nn.Linear(32*10*10, 32*32*32)
		
		self.dropout = nn.Dropout(p = 0.2)
		
	def forward(self, x):
		x1 = self.conv1(x)
		x1 = x1.view(32*32*32, -1)
		
		x2 = self.conv1(x)
		x2 = self.conv2(x2)
		x2 = x2.view(32*32*32, -1)
		
		x3 = self.conv1(x)
		x3 = self.conv3(x3)
		x3 = x3.view(32*32*32, -1)
		
		x4 = self.conv1(x)
		x4 = self.conv2(x4)
		x4 = self.pool4(x4)
		x4 = x4.view(-1, 32*10*10)
		x4 = self.linear4(x4)
		x4 = x4.view(32*32*32, -1)
		
		x = x1 + x2 + x3 + x4
		# print(x.size())
		x = x.view(-1, 32*32*32)
		x = self.linear2(x)
		
		return x
		
		
print ('...')

...


In [142]:
### Choose different CNN architectures here! ###

classifier = TrafficSignClassifier(num_gpu).to(device)
#classifier = TrafficSignClassifier1(num_gpu).to(device)
#classifier = TrafficSignClassifier2(num_gpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (num_gpu > 0):
    classifier = nn.DataParallel(classifier, list(range(num_gpu)))
    
#To load the trained weights
#Uncomment the following line to load the weights. Then you can run testing directly or continue the training
#classfier.load_state_dict(torch.load('./MP2weights.pth', map_location='cpu')) 
#classfier.eval()

#Print the model
print(classifier)

DataParallel(
  (module): TrafficSignClassifier(
    (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv5): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv6): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn5): BatchNorm2

### Set up training environment
You can choose different loss functions and optimizers. Here I just use the same ones as in PyTorch official tutorial.

In [143]:
#training parameters
##TODO
criterion = nn.CrossEntropyLoss()#loss function
optimizer = optim.SGD(classifier.parameters(), lr=0.001, momentum=0.9)
####

Now let's start the training. Just like in tutorial, you can print out your loss and the running time at the end of each epoch to monitor the training process. 

In [144]:
def calculate_val_accuracy(valloader):
    correct = 0.
    total = 0.
    predictions = []

    class_correct = list(0. for i in range(num_classes))
    class_total = list(0. for i in range(num_classes))

    for data in valloader:
        images, labels = data
        if num_gpu > 0:
            images = images.cuda()
            labels = labels.cuda()
        outputs = classifier(Variable(images))
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(list(predicted.cpu().numpy()))
        total += labels.size(0)
        correct += (predicted == labels).sum()

        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i]
            class_total[label] += 1

    class_accuracy = 100.0 * np.divide(class_correct, class_total)
    return 100.0*correct/total, class_accuracy


In [145]:

# Training Loop, no need to run if you already loaded the weights
torch.cuda.empty_cache()
for epoch in range(20):  # loop over the dataset multiple times  
	running_loss = 0.0
	for i, data in enumerate(trainClassifierLoader, 0):
		# get the inputs
		inputs, labels = data
		if num_gpu > 0:
			inputs = inputs.cuda()
			labels = labels.cuda()
	
		# zero out the parameter gradients
		#Every time a variable is back propogated through, the gradient will be accumulated instead of being replaced. 
		optimizer.zero_grad()
	
		# forward + backward + optimize
		outputs = classifier(inputs)
		loss = criterion(outputs, labels)
	
		#loss.backward() computes dloss/dx for every parameter x
		loss.backward()	
	
		#optimizer.step updates the value of x using the gradient x.grad.
		optimizer.step() 
	
		# print statistics
		running_loss += loss.item()
		
		# if i % 100 == 99:    # print every 2000 mini-batches
		# 	print('[%d, %5d] loss: %.3f' %
		# 		(epoch + 1, i + 1, running_loss / 2000))
		# 	running_loss = 0.0
	
	acc, class_acc = calculate_val_accuracy(trainClassifierLoader)
	print('epoch', epoch, ': loss = ', running_loss, ' accuracy = %d %%' % acc)

print('Finished Training')
torch.cuda.empty_cache()


epoch 0 : loss =  758.9375938177109  accuracy = 57 %
epoch 1 : loss =  388.2943978905678  accuracy = 80 %
epoch 2 : loss =  218.08152070641518  accuracy = 90 %
epoch 3 : loss =  135.36966928839684  accuracy = 94 %
epoch 4 : loss =  92.93846182525158  accuracy = 96 %
epoch 5 : loss =  67.65187145769596  accuracy = 97 %
epoch 6 : loss =  52.34990378469229  accuracy = 98 %
epoch 7 : loss =  41.95499961078167  accuracy = 98 %
epoch 8 : loss =  34.24779935181141  accuracy = 99 %
epoch 9 : loss =  28.822373893111944  accuracy = 99 %
epoch 10 : loss =  24.49759256094694  accuracy = 99 %
epoch 11 : loss =  21.211880292743444  accuracy = 99 %
epoch 12 : loss =  18.537507347762585  accuracy = 99 %
epoch 13 : loss =  16.219843819737434  accuracy = 99 %
epoch 14 : loss =  14.21809770166874  accuracy = 99 %
epoch 15 : loss =  12.709333140403032  accuracy = 99 %
epoch 16 : loss =  11.785609509795904  accuracy = 99 %
epoch 17 : loss =  10.38278223015368  accuracy = 99 %
epoch 18 : loss =  9.485080260

### Test the Model
Now our model training is finished. If you are satisfied by the result from part of the test set, let's try it on all testing images. Print out the accuracy on all test images. Note you need to achieve 97% accuracy to get full grade.

In [146]:
test_acc, test_class_acc = calculate_val_accuracy(testClassifierLoader)
print('accuracy = %d %%' % test_acc)

accuracy = 99 %


### Analyze the Results for Each Individual Class
To help futher improve the accuracy, we want to know for which classes our NN works well and for which classes it fails. Print out the accuracy of your classifier on each class on the whole testing dataset. Try to explain why some classess have low accuracy in your report. You don't have to speficy the name of each class. Just use something like "class 0" is good enough. 

In [ ]:
class_correct = list(0. for i in range(num_classes))
class_total = list(0. for i in range(num_classes))
classes = list(range(num_classes))
with torch.no_grad():
    for data in testClassifierLoader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs = classifier(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(labels.size(0)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(num_classes):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

### Save Trained Weights
Notice that the trained weights are just variables right now and will be lost when you close the Jupyter file. Obviously, you don't want to train the model again and again. PyTorch can help you save your work. Read the "Saving & Loading Model for Inference" part from this tutorial: https://pytorch.org/tutorials/beginner/saving_loading_models.html.<br>
Please save the weights from your best model into "MP2weights.pth" and include it in your submission. TAs will not train the CNN for you. So if we cannot find this file, you will lose a lot of points.  

In [36]:
#save the weights into "./MP2weights.pth"
torch.save(classifier.state_dict(), "./cnn_inception.pth")

### Tips for Advanced Techniques
Congratulations! Now we have built a wonderful trffic sign classifier. If you are unsatified about the accuraccy. Here are a few tricks that can help you improve the results:<br>
<li>Balance the dataset. Currently, the size of image dataset from different types of traffic signs vary from 100 to 2000. Hence, our CNN will be trained to be very good at classify those types with a lot of image samples, but behave poorly on the rest of types. One possible solution is to generate more image data from the "unfavorable" types to make each type to have similar number of image samples. </li>
<li>The selection of loss functions and optimizaters  and the learning rate could make a difference. Here is a good reference for different types of loss functions: https://isaacchanghau.github.io/post/loss_functions/</li>
<li>Training the model for too long will make our CNN overfit the training set. If that is the case, we could change the number of epoches to avoid it. </li>
<li>There are piles of articles online that teaches you how to improve your CNN. Do some Google search yourself and try implement some interesting ones.</li>

### Break It If You Can!
Now if you already have trained a neural network with accuracy higher than 90%, let's try to break our system by manipulating the test set. <br>
Find the smallest amount of salt and pepper (recall from lecture 2) noise that has to be added to any image in the data set that was classified correctly, for the image with noise to be miss-classified. Write a function to find this smallest noise for mis-classification, for each image class. Then explore how the smallest noise changes across the different classes. Perform the same experiment with gaussian noise instead of salt and pepper noise. What conclusions can you draw about robustness of your classifier from these experiments? <br>
Note: In this part you only need to pick one single image and test your code on it. Please put the original image and the image after adding noises into your report. 

In [100]:
adv_test_root_sp = 'GTSRB/test_adv/salt_pepper/'
adv_test_root_gaussian = 'GTSRB/test_adv/gaussian/'
adv_test_root = 'GTSRB/test_adv/original'
img_name = '00000_00027.ppm'
# Image to be tested: /home/peixin/mp2_data/Online-Test-sort/00000/00000_00027.ppm

# add s&p or gaussian noise to A SINGLE image
# read_dir: the directory of the image
# write_dir: directory to same the new image
# method: 'sp' or 'gaussian'
# salt & pepper: arg = probability of noise
# gaussian: arg = sigma of gaussian distribution
def add_noise(img_dir, write_dir, method, arg):
    for subdir, dirs, imgs in os.walk(img_dir):
        for img_name in imgs:
            if img_name.endswith('.ppm'):
                img = cv2.imread(os.path.join(subdir, img_name), 1)
                row, col , channel = img.shape
                if method == 'sp':
                    thres = 1 - arg/2

                    for i in range(row):
                        for j in range(col):
                            rand = random.random()
                            # black
                            if rand < arg/2:
                                img[i, j, :] = [0, 0, 0]
                            # white
                            elif rand > thres:
                                img[i, j, :] = [255, 255, 255]
                elif method == 'gaussian':
                    noise = np.random.normal(0, arg, (row, col, channel))
                    noise = np.reshape(noise, (row, col, channel))
                    img = img + noise
                
                cv2.imwrite(os.path.join(write_dir + subdir[23:], img_name), img)#!!!!!Change the 23 value if you change the file paths

    
def calc_class_accuracy(class_list, valloader, classifier_adv):
    class_correct = list(0. for i in range(num_classes))
    class_total = list(0. for i in range(num_classes))
    accuracies = list(0. for i in range(len(class_list)))
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = classifier_adv(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(labels.size(0)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
    j=0
    for class_no in class_list:
        accuracies[j] = 100 * class_correct[class_no] / class_total[class_no]
        j+=1
    return accuracies
    
    
#add_noise(os.path.join(adv_test_root, img_name), os.path.join(adv_test_root_gaussian, 'img.ppm'), 'gaussian', 5)


In [137]:
#chose class 9, 24, and 33 to use for this. Using weights from our 3 layer model (best performing model).
#to decrease runtime, we increment sp value by .05 and gaussian value by 50

# calculate accuracy
classifier_adv = TrafficSignClassifier(num_gpu)
if (device.type == 'cuda') and (num_gpu > 0):
    classifier_adv = nn.DataParallel(classifier_adv, list(range(num_gpu)))
classifier_adv.load_state_dict(torch.load('cnn_3layer.pth'))
classifier_adv.eval()

arg_sp = 0.0
arg_gaus = 300
class_list = [9,24,33]
sp_mins = [0,0,0]
gaus_mins = [0,0,0]

#s+p
while(arg_sp < 1 and any(val == 0 for val in sp_mins)):
    arg_sp += .05
    add_noise(adv_test_root, adv_test_root_sp, 'sp', arg_sp)
    advtestClassifierDataset = dset.ImageFolder(root=adv_test_root_sp,
                               transform=transforms.Compose([
                                   transforms.Resize(image_size),
                                   transforms.CenterCrop(image_size),
                                   transforms.ToTensor(),
                                   # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                   normImgTensor
                               ]))
    advtestClassifierLoader = torch.utils.data.DataLoader(advtestClassifierDataset, batch_size=batch_size, shuffle=True, num_workers=workers)
    sp_acc = calc_class_accuracy(class_list,advtestClassifierLoader,classifier_adv)
    for i, accuracy in enumerate(sp_acc):
        if(sp_mins[i] == 0 and accuracy == 0):
            sp_mins[i] = arg_sp
            print("class: "+str(class_list[i])+" has min argument of: "+str(arg_sp))
    #print(sp_acc,sp2_acc)
print(sp_mins)

#gaus
while(arg_gaus < 2000 and any(val == 0 for val in gaus_mins)):
    arg_gaus += 50
    add_noise(adv_test_root, adv_test_root_gaussian, 'gaussian', arg_gaus)
    advtestClassifierDataset = dset.ImageFolder(root=adv_test_root_gaussian,
                               transform=transforms.Compose([
                                   transforms.Resize(image_size),
                                   transforms.CenterCrop(image_size),
                                   transforms.ToTensor(),
                                   # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                   normImgTensor
                               ]))
    advtestClassifierLoader = torch.utils.data.DataLoader(advtestClassifierDataset, batch_size=batch_size, shuffle=True, num_workers=workers)
    gaus_acc = calc_class_accuracy(class_list,advtestClassifierLoader,classifier_adv)
    for i, accuracy in enumerate(gaus_acc):
        if(gaus_mins[i] == 0 and accuracy == 0):
            gaus_mins[i] = arg_gaus
            print("class: "+str(class_list[i])+" has min argument of: "+str(arg_gaus))

print(gaus_mins)

class: 33 has min argument of: 0.65
class: 24 has min argument of: 0.8000000000000002
class: 9 has min argument of: 0.8500000000000002
[0.8500000000000002, 0.8000000000000002, 0.65]
class: 33 has min argument of: 400
class: 24 has min argument of: 550
class: 9 has min argument of: 950
[950, 550, 400]
